In [1]:
import pdfplumber as pp
import pandas as pd

In [2]:
pdf = pp.open("pdf0116.pdf")

# pdf의 필요한 테이블을 데이터프레임으로 저장

In [3]:
pages = [i for i in range(19,32+1)]
pages = pages + [36, 40, 44, 48, 54, 55, 60,
                64, 69, 74, 79, 80, 86, 87, 
                91, 94, 97, 101, 106, 110, 114,
                120, 121]
print(len(pages), pages)

37 [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 36, 40, 44, 48, 54, 55, 60, 64, 69, 74, 79, 80, 86, 87, 91, 94, 97, 101, 106, 110, 114, 120, 121]


In [4]:
turn = [[1,2], [1,2], [1], [1,2,3], [1,2], [1,2,3],[1,2],
       [1,2],[1,2],[1,2,3,4],[1,2],[1,2],[1,2,3],[1,2,3],
       [1],[1],[1],[1],[1],[1], [1],[1],[1],[1],[1],
       [1],[1],[1],[1],[2],[2], [1],[1],[1],[1],[2],[1]]

# import numpy as np
# test = np.array(turn) -> 왜 안될까
# test.size
cnt = 0
for i in range(len(turn)):
    for j in range(len(turn[i])):
        cnt+=1

print(len(turn), cnt, turn)

37 56 [[1, 2], [1, 2], [1], [1, 2, 3], [1, 2], [1, 2, 3], [1, 2], [1, 2], [1, 2], [1, 2, 3, 4], [1, 2], [1, 2], [1, 2, 3], [1, 2, 3], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [2], [1], [1], [1], [1], [2], [1]]


In [5]:
tables= []
for i, p in enumerate(pages):
    # 해당 페이지에 존재하는 모든 테이블 추출
    page_tables = pdf.pages[p-1].extract_tables()
    #print(type(page_tables)) -> list
    for t in turn[i]:
        tables.append(pd.DataFrame(page_tables[t-1]))

In [6]:
len(tables)

56

# 데이터프레임 정리

### 필요한 컬럼만 남기기

In [9]:
tables[:3]

[   0       1       2        3   4      5   6    7       8       9   \
 0  순번  이수\n구분  개설\n학년     과목코드  분반   교과목명  학점  교수명  학년\n제한  전공\n제한   
 1   1      교필       1  AC00007  01  인권과평화   2  조경희       Y       Y   
 2   2      교필       1  AC00007  02  인권과평화   2  조경희       Y       Y   
 3   3      교필       1  AC00007  03  인권과평화   2  이상윤       Y       Y   
 4   4      교필       1  AC00007  04  인권과평화   2  오영숙       Y       Y   
 5   5      교필       1  AC00007  05  인권과평화   2  강성현       Y       Y   
 6   6      교필       1  AC00007  06  인권과평화   2  강성현       Y       Y   
 
                    10                    11  
 0              강의실/시간                  확인사항  
 1  7207/화/09:00~10:50        주제 : 식민주의/인종주의  
 2  7207/화/11:00~12:50        주제 : 식민주의/인종주의  
 3  7208/화/09:00~10:50   주제 : 인공지능과 인간의 \n공존  
 4  7208/화/11:00~12:50            주제 : 빈곤포르노  
 5  7309/화/09:00~10:50  주제 : 한국의 전쟁과 인권 \n평화  
 6  7309/화/11:00~12:50  주제 : 한국의 전쟁과 인권 \n평화  ,
    0       1       2        3   4     5   6    7    

In [13]:
for i, table in enumerate(tables):
    # tables[i] = table[['이수\n구분','과목코드','교과목명','교수명','강의실/시간','확인사항']]
    tables[i] = table.loc[:, ['이수\n구분','과목코드','교과목명','교수명','강의실/시간','확인사항']]

KeyError: "None of [Index(['이수\\n구분', '과목코드', '교과목명', '교수명', '강의실/시간', '확인사항'], dtype='object')] are in the [columns]"

In [ ]:
tables[:5]

### column 헤더 추출

In [ ]:
# 
header = ['kind','code','classname','prof','roomntime','check']
header

### tables 첫 행 삭제 

In [ ]:
# 
tablecopy = tables.copy()
for i, t in enumerate(tables):
    tablecopy[i] = t.iloc[1:, :]

In [ ]:
# tables[0:5]

### 테이블 하나로 합치기

In [ ]:
# 
df = tablecopy[0]
for t in tablecopy[1:]:
    df = pd.concat([df, t], axis=0)
df[:5]

### 헤더 설정
순번, 이수\n구분, 개설\n학년, 과목코드, 분반, 교과목명, 학점, 교수명, 학년\n제한, 전공\n제한, 강의실/시간, 확인사항

In [ ]:
df.columns = header

### 확인사항 반영(인평, 대생세)

In [ ]:
# 인덱스 초기화
df.reset_index(inplace=True) 

In [ ]:
df1 = df.copy()
subjects = ['인권과평화', '대학생활세미나 I']

for index, row in df1.iterrows():
    if (row.classname in subjects):
        df1.loc[index, 'classname'] = row.classname + '(' + row.check + ')'

In [ ]:
df1.tail()

In [ ]:
df = df1.copy()

### 필요한 컬럼만 추출
index, kind, code, classname, prof, roomntime

In [ ]:
# df = df[['kind','code','classname','prof','roomntime']]
# df

### roomntime 컬럼 분리